#AutoEncoders

##Downloading the dataset

###ML-100K

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-08-28 14:48:26--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  27.8MB/s    in 0.2s    

2021-08-28 14:48:26 (27.8 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2021-08-28 14:48:27--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  33.3MB/s    in 0.2s    

2021-08-28 14:48:27 (33.3 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [4]:
# We won't be using this dataset.
movies = pd.read_csv("ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1")
#columns: id_movies, movie_names, movie_genre
users = pd.read_csv("ml-1m/users.dat", sep="::", header=None, engine="python", encoding="latin-1")
#users column: id_user, gender, age, job code
ratings = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, engine="python", encoding="latin-1")
#ratings column: id_user, id_movie, ratings, timestamps

## Preparing the training set and the test set


In [5]:
# preparing the training and the test datasets
training_set = pd.read_csv("ml-100k/u1.base", delimiter="\t",) #contains user, id_movie, rating, and timestamp
training_set = np.array(training_set, dtype="int") #torch accepts numpy arrays
test_set = pd.read_csv("ml-100k/u1.base", delimiter="\t",) #contains user, id_movie, rating, and timestamp
test_set = np.array(test_set, dtype="int") #torch accepts numpy arrays

## Getting the number of users and movies


In [6]:
#maximum number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

## Converting the data into an array with users in lines and movies in columns


In [7]:
# converting the training and test data into an array with users in rows, movies in columns and ratings in cells
# needed for pytorch, create a list of lists
def convert(data):
    new_data=[]
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1] [data[: ,0] == id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1]=id_ratings #movies not rated are filled with zero
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [8]:
#needed for ML and DL, better to build tensor arrays (more efficient that numpy arrays)
#FloatTensor expects a list of lists
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [9]:
#we will be inheriting our functions and variables for the SAE from nn.Module (parent)
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)   #20 tunable
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
#building an AE with 4 layers: 2 for encoding and 2 for decoding
#specifying the activation function

#encoding, decoding, and applying activation function
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
#creating an object of our SAE class
sae = SAE()
criterion = nn.MSELoss()  #a criterion for training (Mean Squared Error)
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) #lr, wd = tunable
#using RMS rather than Adam
#weight_decay=used to reduce the learning rate after every few epochs in order to regulate the convergenc

## Training the SAE


In [10]:
nb_epoch = 200  #tunable
#building two //for loops//, one for all epochs, the other for all observations(users)
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
#above: creating a single input vector for online learning
#targets = a clone of the input
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False #makes sure we don't compute gradient with respect to target
      output[target == 0] = 0 #dealing with non-zero values only
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)  #for non-nil value
#mean_corrector represents the avverage of the error bbut by only considering the movies that were
#rated (non-zero ratings)
      loss.backward() #calling backward method for the loss
#backward decides the direction of weight update(increase or decrease) 
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step() #optimizer decides the intensity of the update
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7718)
epoch: 2loss: tensor(1.0966)
epoch: 3loss: tensor(1.0534)
epoch: 4loss: tensor(1.0383)
epoch: 5loss: tensor(1.0306)
epoch: 6loss: tensor(1.0269)
epoch: 7loss: tensor(1.0239)
epoch: 8loss: tensor(1.0219)
epoch: 9loss: tensor(1.0210)
epoch: 10loss: tensor(1.0200)
epoch: 11loss: tensor(1.0191)
epoch: 12loss: tensor(1.0185)
epoch: 13loss: tensor(1.0178)
epoch: 14loss: tensor(1.0176)
epoch: 15loss: tensor(1.0174)
epoch: 16loss: tensor(1.0169)
epoch: 17loss: tensor(1.0168)
epoch: 18loss: tensor(1.0164)
epoch: 19loss: tensor(1.0164)
epoch: 20loss: tensor(1.0161)
epoch: 21loss: tensor(1.0158)
epoch: 22loss: tensor(1.0159)
epoch: 23loss: tensor(1.0158)
epoch: 24loss: tensor(1.0155)
epoch: 25loss: tensor(1.0155)
epoch: 26loss: tensor(1.0156)
epoch: 27loss: tensor(1.0153)
epoch: 28loss: tensor(1.0151)
epoch: 29loss: tensor(1.0128)
epoch: 30loss: tensor(1.0111)
epoch: 31loss: tensor(1.0103)
epoch: 32loss: tensor(1.0079)
epoch: 33loss: tensor(1.0080)
epoch: 34loss: tens

## Testing the SAE


In [11]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9139)
